## To-do list
<li>Filter out large gap not alerting field</li>
<li>Find rolling average</li>
<li>Compare each new row with rolling average</li>

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.types import StructType
import json
emailSchema = StructType() \
        .add("metadata", StructType()\
             .add("path",StringType())\
             .add("_attachment_mimetype",StringType())\
             .add("type_prefix",StringType())\
             .add("host",StringType())\
             .add("json",StringType())\
             .add("producer",StringType())\
             .add("topic",StringType())\
             .add("_id",StringType())\
             .add("type",StringType())\
             .add("timestamp",LongType())\
            )\
        .add("data",StructType()\
             .add("code",StringType())\
             .add("system",StringType())\
             .add("uri_path",StringType())\
             .add("method",StringType())\
             .add("clientip",StringType())\
             .add("client",StringType())\
             .add("rec_date",StringType())\
             .add("dn",StringType())\
             .add("api",StringType())\
             .add("rec_timestamp",StringType())\
             .add("frontend",StringType())\
            )\

In [ ]:

# Subscribe to 1 topic

# .option("kafka.bootstrap.servers", "188.185.79.229:9092")\
# .option("startingOffsets", "earliest") \

raw_data = spark.readStream.format("kafka")\
.option("kafka.bootstrap.servers", "monit-kafka.cern.ch:9092")\
.option("subscribe","cmsweb_logs")\
.load()\
.select(\
        from_json(col("value").cast("string"),emailSchema)\
            .getField("metadata").alias("metadata").getField("host").alias("host")\
        ,col("timestamp").alias("timestamp")\
        ,from_json(col("value").cast("string"),emailSchema)\
            .getField("data").alias("data").getField("system").alias("system")\
        ,from_json(col("value").cast("string"),emailSchema)\
            .getField("data").alias("data").getField("dn").alias("user")\
)

raw_data.printSchema()

In [ ]:
groupped_data=raw_data\
.withWatermark("timestamp", "1 minute")\
.groupBy(window('timestamp', "1 minute", "1 minute"),"system")\
.agg(count("system").alias("count"))\

groupped_data.printSchema()

In [ ]:

#function to calculate number of seconds from number of days
days = lambda i: i * 86400
avg_data = groupped_data.withColumn('window.start', groupped_data.window.start.cast('timestamp'))

#create window by casting timestamp to long (number of seconds)
w = (Window.orderBy(col("window.start").cast('long')).rangeBetween(-days(7), 0))

avg_data = avg_data.withColumn('rolling_average',avg("count").over(w))

In [ ]:
raw_data_flow = raw_data.writeStream.queryName("email").outputMode("Append").format("memory").start()
groupped_data_flow = groupped_data.writeStream.queryName("groupped_email").outputMode("Append").format("memory").start()

average_data_flow = average_data.writeStream.queryName("average_email").outputMode("Append").format("memory").start()

In [ ]:
spark.streams.active

In [ ]:
# groupped_data_flow.lastProgress
# raw_data_flow.lastProgress

In [ ]:
raw_data_flow.stop()
groupped_data_flow.stop()

In [ ]:
alerts = spark.sql("select * from email")
alerts.show()

In [ ]:
alerts = spark.sql("select * from groupped_email")
alerts.show()